# Guide to Using Kinetica UDFs

Here we describe how to write and run Kinetica UDFs. We will go through a number of examples using the Python and C++ UDF APIs:

1. UDF using Python API to perform a (CPU-based) mathematical operation
3. UDF using C++ API to access CUDA to perform a (GPU-based) mathematical operations

# 0. Preliminaries #

Start by customizing the following global Python variables, to be used in the examples below:

In [3]:
KINETICA_HOST = 'your_host'
KINETICA_PORT = '9191'
USER_NAME = 'admin'
PASSWORD = '*****'

Next, pull the necessary APIs from GitHub. First, we have Kinetica APIs — of these, we will use only the Kinetica Python API. Second, we will also need the Kinetica-UDF API — in the examples below we will use both the Python and C++ variants. 

In [4]:
%%bash
git clone https://github.com/kineticadb/kinetica-api-python.git ../kinetica-api-python
git clone https://github.com/kineticadb/kinetica-udf-api-python.git ../kinetica-udf-api-python
git clone https://github.com/kineticadb/kinetica-udf-api-cpp.git ../kinetica-udf-api-cpp

fatal: destination path '../kinetica-api-python' already exists and is not an empty directory.
fatal: destination path '../kinetica-udf-api-python' already exists and is not an empty directory.
fatal: destination path '../kinetica-udf-api-cpp' already exists and is not an empty directory.


Next, we will connect to Kinetica:

In [5]:
import sys
sys.path.insert(0,'../kinetica-api-python/gpudb')
import collections

In [6]:
import gpudb
import json
import numpy as np
import pandas as pd
#import tensorflow as tf

h_db = gpudb.GPUdb(encoding = 'BINARY', host = KINETICA_HOST+':'+KINETICA_PORT,  username=USER_NAME, password=PASSWORD)

First, start by creating the table we will fill:

In [7]:
INPUT_TABLE = 'in_table'

source_type = """
{
    "type": "record",
    "name": "source_type",
    "fields": [
            {"name":"x1","type":"float"},
            {"name":"x2","type":"float"}
    ]
}  """ 
    
response = h_db.create_type( type_definition = source_type, label = 'source_type_lbl', properties = {})
type_id = response['type_id']
response = h_db.clear_table( table_name = INPUT_TABLE )
response = h_db.create_table( table_name = INPUT_TABLE, type_id = type_id)
response = h_db.create_table( table_name = INPUT_TABLE, type_id = type_id)

Next we will fill this table with some random data:

In [8]:
import random

encoded_obj_list = []
for val in range(10000):
   datum = collections.OrderedDict()
   datum["x1"] = random.gauss(1,1)
   datum["x2"] = random.gauss(1,2)
   encoded_obj_list.append(h_db.encode_datum(source_type, datum))
response = h_db.insert_records( table_name = INPUT_TABLE,data = encoded_obj_list,list_encoding = 'binary',options = {})

Finally, we will create an (empty) output table, which we will fill by using UDFs:

In [9]:
OUTPUT_TABLE = 'out_table'
out_type = """
{
    "type": "record",
    "name": "out_type",
    "fields": [
            {"name":"y","type":"float"}
    ]
}  """.replace(' ','').replace('\n','')
    
response = h_db.create_type( type_definition = out_type, label = 'out_type_lbl', properties = {})
type_id = response['type_id']
response = h_db.clear_table( table_name = OUTPUT_TABLE)
response = h_db.create_table( table_name = OUTPUT_TABLE, type_id = type_id)

# 1. Basic Python UDF

We will start with a basic Python UDF. 

The following script works by first accessing the data from the API's ProcData(). We assume there is only one input table and one output table, which can therefore be accessed from the API via proc_data.input_data[0] and proc_data.output_data[0]. The next line then performs the calculation: y = x1^2 + x2^2

The final three lines then complete the API call, by loading the out_table structure into Kinetica.

When developing Kinetica UDFs, it is helpful to create a log-file for troubleshooting. Since the UDF is run by the user gpudb_proc, we write this in its home directory:

In [10]:
%%writefile simple_python_udf.py
from kinetica_proc import ProcData
from collections import OrderedDict
import math

#INPUT_TABLE = 'in_table'
#OUTPUT_TABLE = 'out_table'

proc_data = ProcData()

log  = open('/home/gpudb_proc/log_file','w')
log.write('Log file...:')

in_table = proc_data.input_data["in_table"]
x1 = in_table["x1"]
x2 = in_table["x2"]

##call library here

out_table = proc_data.output_data["out_table"]
y = out_table["y"]

log.write('x1 column: ' + x1.name + '\n')
log.write('x2 column: ' + x2.name + '\n')
log.write('y column: ' + y.name + '\n')

out_table.size = in_table.size

for i in xrange(0, in_table.size):
    y[i] = x1[i] ** 2 + x2[i] ** 2

proc_data.complete()

Overwriting simple_python_udf.py


# 2. Creating a Kinetica proc

Having created a UDF file, we can use the /create/proc endpoint to load it as a Kinetica UDF (i.e. proc). We need to specify a proc_name, the file_name(s) to upload, the command and arguments used to run the proc, and the execution mode (i.e. 'distributed' or 'nondistributed').

For the ease of this Jupyter notebook, start by specifying the following options:

In [11]:
proc_name = 'simple_python_udf'
proc_language = 'python'
execution_mode = 'distributed'

In [12]:
if proc_language=='python':
   file_names = [proc_name + '.py']
   command = 'python'
   args=[file_names[0]]
elif proc_language=='cpp':
   file_names = [proc_name]
   command = './' + proc_name
   args=['0']
else:
   raise Exception('Unknown language, set above')
options={}

files = {}

for file_name in file_names:
    bytes = ''
    file = open(file_name, 'rb')
    while True:
        chunk = file.read(1024)  
        if not chunk:
            break
        bytes = bytes + chunk
    files[file_name] = bytes
    file.close()
        
    
if h_db.has_proc(proc_name)['proc_exists']:
    response = h_db.delete_proc(proc_name)

response = h_db.create_proc(proc_name, execution_mode, files, command, args, options)
print response

OrderedDict([(u'proc_name', u'simple_python_udf'), ('status_info', OrderedDict([(u'status', u'OK'), (u'message', u''), (u'data_type', u'create_proc_response'), ('response_time', 0.0851)]))])


In [14]:
print h_db.has_proc(proc_name)

OrderedDict([(u'proc_name', u'simple_python_udf'), (u'proc_exists', True), ('status_info', OrderedDict([(u'status', u'OK'), (u'message', u''), (u'data_type', u'has_proc_response'), ('response_time', 0.00014)]))])


# 3. Executing a Kinetica proc

Now that the proc has been created, it can subsequently be executed via the /execute/proc endpoint. For example, using the Python API:

In [73]:
params = {}
bin_params = {}
input_table_names = [INPUT_TABLE]
input_column_names = {}
output_table_names = [OUTPUT_TABLE]
options = {} 

response = h_db.execute_proc(proc_name, params, bin_params, input_table_names, input_column_names, output_table_names, options )
print response

OrderedDict([(u'run_id', u'2'), ('status_info', OrderedDict([(u'status', u'OK'), (u'message', u''), (u'data_type', u'execute_proc_response'), ('response_time', 0.01786)]))])


## 4. Output
Show output of proc (out_table)

In [83]:
response = h_db.get_records('out_table', 0, 10,'json', {})
od = OrderedDict(response)
od.keys()
od['records_json']


[]